# Train KIOptipack Model on Apheris

### Load environment variables
Load environment variables to connect with Apheris APIs.

In [1]:
import dotenv
dotenv.load_dotenv(PATH_TO_ENV_FILE)

True

### Import Apheris API modules
Import required modules for dataset access, compute, model management, and authentication.

In [2]:
from aphcli.api import datasets
from aphcli.api import compute, models, job
from apheris import login, logout
from apheris_auth.remote_data import RemoteData

### Log in to Apheris platform
Authenticate your session using Apheris login.

In [3]:
login()

You are already logged in


### List available datasets
Retrieve and display the datasets available to the current user.

In [4]:
datasets.list_datasets()

idx,dataset_id,organization,data custodian
0,task004-hippocampus-set-a_apheris-gateway-1_demo-1,Demo 1,Apheris Demo1
1,sherman-2016-tumor-survival-a_apheris-gateway-1_demo-1,Demo 1,Apheris Demo1
2,sherman-2016-tumor-survival-b_apheris-gateway-2_demo-2,Demo 2,Apheris Demo2
3,sherman-2016-tumor-survival-c_apheris-gateway-1_demo-1,Demo 1,Apheris Demo1
4,omop-ehr-hospital-1_apheris-gateway-1_demo-1,Demo 1,Apheris Demo1
5,omop-ehr-hospital-2_apheris-gateway-2_demo-2,Demo 2,Apheris Demo2
6,kiodatasetv1_kio_rwthaachen_kio_rwthaachen,kio_rwthaachen,Alexander Nasuta
7,whas2_apheris-gateway-2_demo-2,Demo 2,Apheris Demo2
8,kio-test-dataset_kio_rwthaachen_kio_rwthaachen,kio_rwthaachen,Sabrina Steinert
9,kio-test-dataset_apheris-gateway-1_demo-1,Demo 1,Apheris Demo1


### Get dataset description
Fetch a detailed description of a specific dataset.

In [5]:
datasets.get_description("kiodatasetv2_kio_rwthaachen_kio_rwthaachen")

{'slug': 'kiodatasetv2_kio_rwthaachen_kio_rwthaachen',
 'name': 'KioDatasetV2',
 'description': 'The dataset contains labeled datapoints related to material validation processes for thermoforming equipment. Each datapoint reflects a specific use case or validation scenario, including information on the material used, equipment configuration, validation method, and outcome.\nThermoforming machine manufacturers typically offer a predefined material portfolio aligned with their equipment specifications. Converters can select suitable materials based on the intended application. To ensure contractual performance targets are met, manufacturers validate the machine\'s efficiency using standard materials. Using alternative materials is generally at the converter\'s own risk. However, some manufacturers support formal validation of unapproved materials—either through practical testing on identical or comparable machines or via structured methods such as Design of Experiments (DOE). These valid

### List available models
Display the models that can be used for compute jobs.

In [6]:
models.list_models()

idx,name,version
0,apheris-gwas,0.0.51
1,apheris-gwas,0.0.50
2,apheris-gwas,0.0.49
3,apheris-gwas-regression,0.0.47
4,apheris-gwas-regression,0.0.46
5,apheris-gwas-regression,0.0.45
6,apheris-hf-tiny-bert,0.2.0
7,apheris-hf-tiny-bert,0.6.1
8,apheris-kioptipack,0.3.4-debug
9,apheris-kioptipack,0.4.0


### Create compute specification
Define and create the compute environment for running the model.

In [7]:
compute_spec_id = compute.create_from_args(
    dataset_ids=["kiodatasetv2_kio_rwthaachen_kio_rwthaachen"],
    model_id='apheris-kioptipack',
    model_version="0.4.0",
    client_memory=1024,
    client_n_cpu=1,
    client_n_gpu=0,
    server_memory=1024,
    server_n_cpu=1,
    server_n_gpu=0,
)

### Activate compute spec
Start the compute environment to prepare for job submission.

In [8]:
compute.activate(compute_spec_id)

### Wait until compute is running
Block until the compute spec is fully operational, then get its status.

In [9]:
compute.wait_until_running(compute_spec_id)
compute.get_status(compute_spec_id)

{'status': 'running',
 'message': '{"f5b4b6cf-6e2d-4cc5-92af-6d7792ead047":"The computation has been scheduled","orchestrator":"The computation has been scheduled"}',
 'details': '{"f5b4b6cf-6e2d-4cc5-92af-6d7792ead047":"The computation is running.","orchestrator":"The computation is running."}'}

### Submit job for model training
Send a job to the compute environment using the specified dataset and columns.

In [10]:
dataset_ids=["kiodatasetv2_kio_rwthaachen_kio_rwthaachen"]
payload = {"dataset_ids": dataset_ids, "prediction_columns": ["Temp", "Zeit", "Druck"]}

job.submit( payload,
    compute_spec_id=compute_spec_id, verbose=True
)


We successfully submitted the job. Please note the ID {id}


UUID('8e791ee7-b8bf-4488-8efc-1d212e94d67b')

### Wait for job completion
Block until the training job has finished and show the final job status.

In [11]:
job.wait_until_job_finished()
job.status()

Using the cached `job_id` 8e791ee7-b8bf-4488-8efc-1d212e94d67b [stored 2025-07-11 17:25:44].
Using the cached `job_id` 8e791ee7-b8bf-4488-8efc-1d212e94d67b [stored 2025-07-11 17:26:48].


'finished.completed'

### Display job logs
Print logs to debug or inspect job output.

In [12]:
print(job.logs())

Using the cached `compute_spec_id` 6afad1cc-1a20-4512-b6bf-86a67e281d77 [stored 2025-07-11 17:25:43].
Using the cached `job_id` 8e791ee7-b8bf-4488-8efc-1d212e94d67b [stored 2025-07-11 17:26:48].
2025-07-11 15:25:45,569 - runner_process - INFO - Runner_process started.
2025-07-11 15:25:45,575 - nvflare.private.fed.app.deployer.server_deployer.ServerDeployer - INFO - server heartbeat timeout set to 600
2025-07-11 15:25:45,679 - CoreCell - INFO - server.8e791ee7-b8bf-4488-8efc-1d212e94d67b: created backbone internal connector to tcp://localhost:44435 on parent
2025-07-11 15:25:45,679 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 ACTIVE tcp://localhost:44435] is starting
2025-07-11 15:25:45,680 - Cell - INFO - Register blob CB for channel='server_command', topic='*'
2025-07-11 15:25:45,680 - Cell - INFO - Register blob CB for channel='aux_communication', topic='*'
2025-07-11 15:25:45,680 - ServerCommandAgent - INFO - ServerCommandAgent cell register_request_cb: server.8e79

## Download Results
Once the job has completed the model weight are available for download from the compute_spec.

In [13]:
job.download_results("/tmp/kioptipack20250711")

Using the cached `compute_spec_id` 6afad1cc-1a20-4512-b6bf-86a67e281d77 [stored 2025-07-11 17:25:43].
Using the cached `job_id` 8e791ee7-b8bf-4488-8efc-1d212e94d67b [stored 2025-07-11 17:26:48].


In [14]:
!ls -lh /tmp/kioptipack20250711/workspace/app_server/

total 64K
drwxrwxr-x 2 linux-evelyn linux-evelyn  60 Jul 11 17:26 config
-rw-r--r-- 1 linux-evelyn linux-evelyn 64K Jul 11 17:26 kioptipack.pt


### Deactivate compute environment
Shut down the compute spec to free up resources and save cost.

In [15]:
compute.deactivate()

Using the cached `compute_spec_id` 6afad1cc-1a20-4512-b6bf-86a67e281d77 [stored 2025-07-11 17:25:43].
